In [2]:
# module
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import numpy as np

In [3]:
# Util Functions
from enum import Enum

# 파일을 읽어와 pandas의 DataFrame으로 반환해줌
def get_df_from(path: str, sep: str = ','):
    extension = path.split('.')[-1]
    try:
        if extension == "csv" or extension == "txt":
            df = pd.read_csv(path, sep=sep, on_bad_lines='skip')
            if df.empty:
                print("W: DataFrame is empty.")
                return None
            return df
        else:
            print("E: File extension is not supported.")
            return None
    except FileNotFoundError:
        print(f"E: File not found. Check the path: {path}")
        return None
    except pd.errors.ParserError as e:
        print(f"E: Error parsing CSV file: {e}")
        return None
    except Exception as e:
        print(f"E: An unexpected error occured: {e}")
        return None

# 리스트의 평균을 반환.(기본값 0)
def calculate_mean(lst):
    if lst: # 빈 리스트가 아닌 경우에만 평균 계산
        return np.mean(lst)
    return 0 # 빈 리스트인 경우 NaN 반환

# 리스트의 합을 반환.(기본값 0)
def calculate_sum(lst):
    if lst:
        return np.sum(lst)
    return 0


class ChartShape(Enum):
    BAR = 1
    HISTOGRAM = 2
    LINE = 3
    HEATMAP = 4

# Chart 뽑는 함수
def get_chart_from(df: pd.DataFrame, x_col: str, y_col: str, shape: ChartShape = ChartShape.BAR, real_time: bool = False):
    plt.figure(figsize=(10, 6))
    if shape == ChartShape.BAR:
        plt.bar(df[x_col], df[y_col])
    elif shape == ChartShape.HISTOGRAM:
        plt.hist(df[x_col], bins=10)
    elif shape == ChartShape.LINE:
        plt.plot(df[x_col], df[y_col])
    else:
        plt.plot(df[x_col], df[y_col])

    if real_time:
        plt.gca().xaxis.set_major_locator(mdates.MonthLocator()) # 월 단위로 큰 눈금 표시
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d')) # 날짜 형식 지정
    plt.xlabel(x_col)
    plt.ylabel(y_col)
    plt.grid(True)
    plt.tight_layout()
    plt.xticks(df[x_col], rotation=45, ha='right')
    plt.show()

def get_chart_from_series(sr: pd.Series, x_col: str, y_col: str, shape: ChartShape = ChartShape.BAR):
    if shape == ChartShape.BAR:
        sr.plot.bar()
    if shape == ChartShape.LINE:
        sr.plot.line()
    if shape == ChartShape.HISTOGRAM:
        sr.plot.hist()
    plt.xlabel(x_col)
    plt.ylabel(y_col)
    plt.show()

def get_integer(question: str):
    try:
        id = int(input(question))
        return id
    except:
        print(f"E: invalid integer")
        return 0

In [4]:
# Function

folder_path = '../public/kmrd-small/'

file_paths = {
    'castings': folder_path + 'castings.csv',
    'countries': folder_path + 'countries.csv',
    'genres': folder_path + 'genres.csv',
    'movies': folder_path + 'movies.txt',
    'peoples': folder_path + 'peoples.txt',
    'rates': folder_path + 'rates.csv',
}

def get_df_strict_from(path: str, sep: str = ','):
    df = get_df_from(path, sep)
    for _ in range(3):
        if df is not None:
            return df
        new_path = input('오류가 발생했습니다. 정확한 경로를 확인해주세요: ')
        new_sep = input('구분자를 입력해주세요: ')
        df = get_df_from(new_path, new_sep)
    exit()

### 정밀도와 재현율

- 사용자별 평가를 10개만 한 사용자 && 영화별 평점 개수가 [60, 200)인 영화

In [5]:
dfs = {}
names = ['rates', 'movies']
for name in names:
    path = file_paths[name]
    if name == 'movies' or name == 'peoples':
        dfs[name] = get_df_strict_from(path, '\t')
    else:
        dfs[name] = get_df_strict_from(path)

In [23]:
from sklearn.preprocessing import MultiLabelBinarizer

In [37]:
rates_df = dfs['rates']
movies_df = dfs['movies']

user_rates_count = rates_df.value_counts('user')
movie_rates_count = rates_df.value_counts('movie')
user_rates_10 = rates_df.groupby('user')['movie'].apply(list)[user_rates_count == 10]

print(user_rates_10[30:40])
movie_rates_60to200 = movie_rates_count[(movie_rates_count >= 60) & (movie_rates_count < 200)].index

result = pd.DataFrame(user_rates_10.apply(lambda x: [item for item in x if item in movie_rates_60to200]))

result.columns = ['movie']
result['cnt'] = result['movie'].apply(len)

print(result[result['cnt'] == 10])

mlb = MultiLabelBinarizer()

movie_mat = pd.DataFrame(mlb.fit_transform(result['movie']), columns=mlb.classes_, index=result.index)

print(movie_mat[49])
# result.pivot_table(index="user", columns='movie', aggfunc='count')

user
1878    [10001, 10102, 10128, 10253, 10453, 10630, 106...
1883    [10001, 10002, 10003, 10004, 10018, 10102, 102...
1887    [10001, 10002, 10102, 10111, 10200, 10244, 103...
1917    [10001, 10046, 10048, 10071, 10102, 10110, 101...
1980    [10001, 10016, 10028, 10039, 10071, 10102, 101...
2010    [10001, 10002, 10021, 10022, 10037, 10173, 102...
2027    [10001, 10018, 10132, 10173, 10472, 10610, 106...
2067    [10001, 10016, 10072, 10086, 10110, 10181, 103...
2089    [10001, 10033, 10034, 10035, 10155, 10156, 101...
2116    [10001, 10035, 10058, 10071, 10114, 10323, 104...
Name: movie, dtype: object
Empty DataFrame
Columns: [movie, cnt]
Index: []


KeyError: 49